In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python-headless

Note: you may need to restart the kernel to use updated packages.


### PART 1. Data preprocessing and visualization

In [7]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

2024-04-10 14:17:54.253585: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 14:17:54.299785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 14:17:54.299846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 14:17:54.302656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 14:17:54.315781: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 14:17:54.316633: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [1]:
import shutil
try:
    shutil.rmtree("data_all_modified/train")
    shutil.rmtree("data_all_modified/test")
except:
    pass

In [2]:

import os
# ensure directories exist
from pathlib import Path
Path("data_all_modified-split/train/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-split/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data_all_modified-split/test/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-split/test/no_damage").mkdir(parents=True, exist_ok=True)

In [3]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above
all_damage_file_paths = os.listdir('data_all_modified/damage')
all_no_damage_file_paths = os.listdir('data_all_modified/no_damage')

In [4]:
import random

train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))

train damage image count:  11336
test damage image count:  2834
len of overlap:  0
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0


In [5]:
# ensure to copy the images to the directories
import shutil
for p in train_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-split/train/damage', p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-split/test/damage', p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-split/train/no_damage', p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-split/test/no_damage', p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("data_all_modified-split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("data_all_modified-split/train/no_damage")))

print("Files in test/damage: ", len(os.listdir("data_all_modified-split/train/damage")))
print("Files in test/no_damage: ", len(os.listdir("data_all_modified-split/train/no_damage")))

Files in train/damage:  14170
Files in train/no_damage:  7148
Files in test/damage:  14170
Files in test/no_damage:  7148


In [11]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

train_data_dir = 'data_all_modified-split/train'

batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 21315 files belonging to 2 classes.
Using 17052 files for training.
Using 4263 files for validation.


In [8]:
test_data_dir = 'data_all_modified-split/test'

batch_size = 2
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 17653 files belonging to 2 classes.


### PART 2. Model design, training, and evaluation. Exploring different model architectures, and identifying the model with high accuracy and validation accuracy.  

### Building A Dence ANN

In [9]:
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

# Intializing a sequential model
model_cnn = models.Sequential()

# Adding first conv layer with 64 filters and kernel size 3x3 , padding 'same' provides the output size same as the input size
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(128,128,3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling to make it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with * neurons and activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(2, activation='softmax'))

# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)        0

In [12]:
#fit the model from image generator
history = model_cnn.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
533/533 [==============================] - 130s 242ms/step - loss: 0.5121 - accuracy: 0.7630 - val_loss: 0.5906 - val_accuracy: 0.7157
Epoch 2/20
533/533 [==============================] - 131s 245ms/step - loss: 0.3822 - accuracy: 0.8445 - val_loss: 0.3290 - val_accuracy: 0.8702
Epoch 3/20
533/533 [==============================] - 144s 270ms/step - loss: 0.3008 - accuracy: 0.8819 - val_loss: 0.2388 - val_accuracy: 0.9082
Epoch 4/20
533/533 [==============================] - 140s 263ms/step - loss: 0.2327 - accuracy: 0.9116 - val_loss: 0.1928 - val_accuracy: 0.9253
Epoch 5/20
533/533 [==============================] - 137s 257ms/step - loss: 0.1833 - accuracy: 0.9296 - val_loss: 0.2382 - val_accuracy: 0.9089
Epoch 6/20
533/533 [==============================] - 134s 251ms/step - loss: 0.1552 - accuracy: 0.9418 - val_loss: 0.1485 - val_accuracy: 0.9408
Epoch 7/20
533/533 [==============================] - 135s 253ms/step - loss: 0.1378 - accuracy: 0.9478 - val_loss: 0.3189 -

In [13]:
test_loss, test_accuracy = model_cnn.evaluate(test_rescale_ds, verbose=0)
test_accuracy

0.5720354318618774

### The Lenet-5 CNN architecture

In [15]:
from keras import layers
from keras import models
import pandas as pd
from tensorflow.keras import optimizers
from tensorflow.keras import layers, models, optimizers
model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_lenet5.add(layers.Dense(3, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d (Average  (None, 63, 63, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_1 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_1 (Flatten)         (None, 14400)             0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [16]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
533/533 [==============================] - 32s 55ms/step - loss: 0.6258 - accuracy: 0.6806 - val_loss: 0.5822 - val_accuracy: 0.6782
Epoch 2/20
533/533 [==============================] - 30s 57ms/step - loss: 0.5373 - accuracy: 0.7465 - val_loss: 0.5344 - val_accuracy: 0.7054
Epoch 3/20
533/533 [==============================] - 31s 57ms/step - loss: 0.4623 - accuracy: 0.7930 - val_loss: 0.4261 - val_accuracy: 0.8213
Epoch 4/20
533/533 [==============================] - 31s 58ms/step - loss: 0.4328 - accuracy: 0.8144 - val_loss: 0.4249 - val_accuracy: 0.8424
Epoch 5/20
533/533 [==============================] - 32s 59ms/step - loss: 0.4149 - accuracy: 0.8228 - val_loss: 0.4300 - val_accuracy: 0.8374
Epoch 6/20
533/533 [==============================] - 33s 61ms/step - loss: 0.3951 - accuracy: 0.8374 - val_loss: 0.3784 - val_accuracy: 0.8506
Epoch 7/20
533/533 [==============================] - 28s 53ms/step - loss: 0.3751 - accuracy: 0.8498 - val_loss: 0.3671 - val_accuracy:

In [30]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose=0)
test_accuracy

0.9232379794120789

### Alternate-Lenet-5 CNN Architecture

In [28]:
model_Altlenet5 = models.Sequential()
# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_Altlenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_Altlenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_Altlenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 32 neurons
model_Altlenet5.add(layers.Dense(512, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_Altlenet5.add(layers.Dense(2, activation='softmax'))

# Compile model
model_Altlenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_Altlenet5.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_35 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_36 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 14, 14, 128)      

In [29]:
history = model_Altlenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
533/533 [==============================] - 110s 205ms/step - loss: 0.4890 - accuracy: 0.7764 - val_loss: 0.4723 - val_accuracy: 0.7833
Epoch 2/20
533/533 [==============================] - 106s 199ms/step - loss: 0.3274 - accuracy: 0.8679 - val_loss: 0.4917 - val_accuracy: 0.7795
Epoch 3/20
533/533 [==============================] - 109s 204ms/step - loss: 0.2263 - accuracy: 0.9099 - val_loss: 0.1809 - val_accuracy: 0.9315
Epoch 4/20
533/533 [==============================] - 107s 201ms/step - loss: 0.1723 - accuracy: 0.9301 - val_loss: 0.1670 - val_accuracy: 0.9348
Epoch 5/20
533/533 [==============================] - 107s 201ms/step - loss: 0.1379 - accuracy: 0.9442 - val_loss: 0.1190 - val_accuracy: 0.9524
Epoch 6/20
533/533 [==============================] - 110s 206ms/step - loss: 0.1219 - accuracy: 0.9515 - val_loss: 0.1281 - val_accuracy: 0.9479
Epoch 7/20
533/533 [==============================] - 106s 198ms/step - loss: 0.1074 - accuracy: 0.9571 - val_loss: 0.1092 -

In [32]:
test_loss, test_accuracy = model_Altlenet5.evaluate(test_rescale_ds, verbose=0)
test_accuracy

0.9901878833770752